In [ ]:
import vectorbt as vbt
import pandas as pd
from vnstock import Vnstock 
import datetime as dt
import talib  # Để tính toán MACD và Signal Line

# Lấy dữ liệu cổ phiếu
vnstock_instance = Vnstock().stock(symbol='ACB', source='TCBS')

stock = "HPG"
end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (dt.datetime.now() - dt.timedelta(days=7000)).strftime('%Y-%m-%d')
data = vnstock_instance.quote.history(symbol=stock, start=start_date, end=end_date, interval='1D')

data.set_index("time", inplace=True)
data.index = pd.to_datetime(data.index)

# Tính MACD và Signal Line
macd, signal, _ = talib.MACD(data['close'], fastperiod=12, slowperiod=26, signalperiod=9)
data['macd'] = macd
data['signal'] = signal

# Entry & Exit tín hiệu từ MACD
entries = macd > signal
exits = macd < signal

# Tạm tạo portfolio để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy giá entry từ các lệnh mua
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[
    portfolio_tmp.orders.records_readable['Side'] == 'Buy'
]
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Stop Loss và Take Profit
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.10  # 10%

stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

stop_loss_signal = data['close'] < stop_loss_level
take_profit_signal = data['close'] > take_profit_level

# Tín hiệu exit cuối cùng
final_exits = exits | stop_loss_signal | take_profit_signal

# Portfolio cuối cùng với SL/TP
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()

# Xem MACD
print("\nMACD và Signal Line:")
print(data[['macd', 'signal']].tail())

# Lưu kết quả
portfolio.stats().to_csv('macd_sl_tp.csv')
print("Chiến lược MACD + Stop Loss + Take Profit đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 

# Khởi tạo và lấy dữ liệu
vnstock_instance = Vnstock().stock(symbol='ACB', source='TCBS')
stock = "HPG"
end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (dt.datetime.now() - dt.timedelta(days=7000)).strftime('%Y-%m-%d')
data = vnstock_instance.quote.history(symbol=stock, start=start_date, end=end_date, interval='1D')
data.set_index("time", inplace=True)
data.index = pd.to_datetime(data.index)

# Tính RSI
rsi = vbt.RSI.run(data['close'], window=14)

# Tín hiệu mua/bán theo RSI
entries = rsi.rsi < 30
exits = rsi.rsi > 70

# Tạo portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy giá entry từ lệnh mua
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[
    portfolio_tmp.orders.records_readable['Side'] == 'Buy'
]
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Thiết lập Stop Loss và Take Profit
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.10  # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu thoát lệnh do SL hoặc TP
stop_loss_signal = data['close'] < stop_loss_level
take_profit_signal = data['close'] > take_profit_level

# Kết hợp với exit gốc (RSI)
final_exits = exits | stop_loss_signal | take_profit_signal

# Portfolio cuối cùng có SL/TP
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('rsi_30_70_sl_tp.csv')
print("Chiến lược RSI + SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 

# Tính Bollinger Bands thủ công
window = 20
std = 2
rolling_mean = data['close'].rolling(window=window).mean()
rolling_std = data['close'].rolling(window=window).std()
bb_upper = rolling_mean + std * rolling_std
bb_lower = rolling_mean - std * rolling_std

# Tín hiệu mua: close từ trên xuống dưới lower band
entries = (data['close'].shift(1) > bb_lower.shift(1)) & (data['close'] < bb_lower)
# Tín hiệu bán: close từ dưới lên trên upper band
exits = (data['close'].shift(1) < bb_upper.shift(1)) & (data['close'] > bb_upper)

# Portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy giá mua
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[
    portfolio_tmp.orders.records_readable['Side'] == 'Buy'
]
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Mức SL/TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu SL/TP
stop_loss_signal = data['close'] < stop_loss_level
take_profit_signal = data['close'] > take_profit_level

# Exit kết hợp
final_exits = exits | stop_loss_signal | take_profit_signal

# Portfolio cuối cùng
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('bb_sl_tp.csv')
print("Chiến lược Bollinger Bands + SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 



# Tính MA20 và độ lệch chuẩn
ma = vbt.MA.run(data['close'], window=20).ma
std = data['close'].rolling(window=20).std()

# Tín hiệu mua/bán
entries = data['close'] < (ma - 1.5 * std)
exits = data['close'] > (ma + 1.5 * std)

# Portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy giá entry
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[
    portfolio_tmp.orders.records_readable['Side'] == 'Buy'
]
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Mức SL/TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu thoát SL/TP
stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Tổng hợp tín hiệu thoát
final_exits = exits | stop_loss | take_profit

# Portfolio chính thức
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('ma_std_sl_tp.csv')
print("Chiến lược MA ± 1.5*STD với SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 


# Tính Stochastic Oscillator
fastk_period = 14
slowk_period = 3
slowd_period = 3

fastk = (data['close'] - data['low'].rolling(fastk_period).min()) / (data['high'].rolling(fastk_period).max() - data['low'].rolling(fastk_period).min()) * 100
slowk = fastk.rolling(slowk_period).mean()
slowd = slowk.rolling(slowd_period).mean()

# Tín hiệu
entries = slowk > 20
exits = slowk < 80

# Portfolio tạm để lấy giá entry
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy entry price
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[portfolio_tmp.orders.records_readable['Side'] == 'Buy']
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Tính SL và TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Kết hợp thoát lệnh
final_exits = exits | stop_loss | take_profit

# Portfolio chính thức
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('stochastic_sl_tp.csv')
print("Chiến lược Stochastic với SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 

# Lấy dữ liệu

# Lấy dữ liệu cổ phiếu


# Tính MA của volume
volume_ma = vbt.MA.run(data['volume'], window=20).ma

# Tín hiệu mua/bán
entries = (data['close'] > data['close'].shift(1)) & (data['volume'] > volume_ma * 1.5)
exits = (data['close'] < data['close'].shift(1)) & (data['volume'] < volume_ma)

# Portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy entry price
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[portfolio_tmp.orders.records_readable['Side'] == 'Buy']
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Tính SL và TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu Stop Loss và Take Profit
stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Kết hợp thoát lệnh
final_exits = exits | stop_loss | take_profit

# Portfolio chính thức
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('price_volume_sl_tp.csv')
print("Chiến lược Price-Volume với SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock


# Tính Parabolic SAR (PSAR)
def calculate_psar(high, low, close, acceleration_factor=0.02, max_acceleration=0.2):
    psar = pd.Series(index=close.index)
    psar.iloc[0] = close.iloc[0]  # Khởi tạo PSAR tại giá đóng cửa của ngày đầu tiên
    
    # Biến trạng thái: xu hướng tăng (True) hoặc giảm (False)
    is_uptrend = True
    ep = high.iloc[0]  # Extreme Point (EP), giá cao nhất nếu xu hướng tăng, thấp nhất nếu giảm
    af = acceleration_factor
    
    for t in range(1, len(close)):
        if is_uptrend:
            psar.iloc[t] = psar.iloc[t-1] + af * (ep - psar.iloc[t-1])
            if close.iloc[t] > ep:  # Cập nhật Extreme Point nếu giá đóng cửa cao hơn EP
                ep = high.iloc[t]
                af = min(af + acceleration_factor, max_acceleration)
            if close.iloc[t] < psar.iloc[t]:  # Đảo chiều xu hướng nếu giá đóng cửa dưới PSAR
                is_uptrend = False
                psar.iloc[t] = ep  # Cập nhật PSAR cho xu hướng giảm
                ep = low.iloc[t]
                af = acceleration_factor
        else:
            psar.iloc[t] = psar.iloc[t-1] + af * (ep - psar.iloc[t-1])
            if close.iloc[t] < ep:  # Cập nhật Extreme Point nếu giá đóng cửa thấp hơn EP
                ep = low.iloc[t]
                af = min(af + acceleration_factor, max_acceleration)
            if close.iloc[t] > psar.iloc[t]:  # Đảo chiều xu hướng nếu giá đóng cửa trên PSAR
                is_uptrend = True
                psar.iloc[t] = ep  # Cập nhật PSAR cho xu hướng tăng
                ep = high.iloc[t]
                af = acceleration_factor
    
    return psar

# Tính PSAR cho dữ liệu
psar = calculate_psar(data['high'], data['low'], data['close'])

# Tín hiệu mua/bán
entries = (data['close'] > psar) & (data['close'].shift(1) <= psar.shift(1))  # Giá vượt lên PSAR -> Mua
exits = (data['close'] < psar) & (data['close'].shift(1) >= psar.shift(1))    # Giá xuống dưới PSAR -> Bán

# Portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy entry price
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[portfolio_tmp.orders.records_readable['Side'] == 'Buy']
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Tính SL và TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu Stop Loss và Take Profit
stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Kết hợp tín hiệu thoát lệnh
final_exits = exits | stop_loss | take_profit

# Portfolio chính thức
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('psar_sl_tp.csv')
print("Chiến lược PSAR với SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock



# Tính VWAP thủ công
def calculate_vwap(high, low, close, volume):
    # Tính giá trung bình (mid-price)
    mid_price = (high + low + close) / 3
    # Tính VWAP
    vwap = (mid_price * volume).cumsum() / volume.cumsum()
    return vwap

# Tính VWAP cho dữ liệu
vwap = calculate_vwap(data['high'], data['low'], data['close'], data['volume'])

# Tín hiệu mua/bán
entries = data['close'] > vwap  # Giá vượt lên VWAP -> Mua
exits = data['close'] < vwap    # Giá xuống dưới VWAP -> Bán

# Portfolio tạm để lấy entry price
portfolio_tmp = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy entry price
entry_price = pd.Series(index=data.index, dtype=float)
buy_orders = portfolio_tmp.orders.records_readable[portfolio_tmp.orders.records_readable['Side'] == 'Buy']
for _, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Tính SL và TP
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu Stop Loss và Take Profit
stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Kết hợp tín hiệu thoát lệnh
final_exits = exits | stop_loss | take_profit

# Portfolio chính thức
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('vwap_sl_tp.csv')
print("Chiến lược VWAP với SL/TP đã được lưu vào file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock 

# Lấy dữ liệu


# Tính ATR (Average True Range) cho trailing stop
atr = vbt.ATR.run(high=data['high'], low=data['low'], close=data['close'], window=14)

# Tính trailing stop: Giá đóng cửa - 2 lần ATR
trailing_stop = data['close'] - (atr.atr * 2)

# Đặt các mức Stop Loss và Take Profit (5% dưới và 10% trên giá mua)
stop_loss_pct = 0.05  # 5% dưới giá mua
take_profit_pct = 0.10  # 10% trên giá mua

# Tín hiệu mua: khi giá đóng cửa vượt lên trên MA20
entries = data['close'] > data['close'].rolling(20).mean()

# Tín hiệu bán: khi giá đóng cửa chạm vào hoặc vượt qua trailing stop hoặc đạt Stop Loss/Take Profit
exits = (
    (data['close'] < trailing_stop) |  # Chạm vào trailing stop
    (data['close'] < data['close'].shift(1) * (1 - stop_loss_pct)) |  # Stop Loss
    (data['close'] > data['close'].shift(1) * (1 + take_profit_pct))  # Take Profit
)

# Tạo danh mục đầu tư từ tín hiệu
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Kết quả
print(portfolio.stats())
portfolio.plot().show()
portfolio.stats().to_csv('atr_with_sl_tp.csv')
print("Các kết quả đã được lưu vào các file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock



# Tính EMA thủ công
def calculate_ema(data, window):
    alpha = 2 / (window + 1)
    ema = pd.Series(index=data.index)
    ema.iloc[0] = data.iloc[0]  # Khởi tạo EMA
    for t in range(1, len(data)):
        ema.iloc[t] = alpha * data.iloc[t] + (1 - alpha) * ema.iloc[t - 1]
    return ema

# Tính EMA ngắn hạn (EMA9) và dài hạn (EMA21)
short_window = 9
long_window = 21
data['ema_short'] = calculate_ema(data['close'], short_window)
data['ema_long'] = calculate_ema(data['close'], long_window)

# Tính Stop Loss và Take Profit
take_profit_pct = 0.1  # Chốt lời khi giá tăng 5% so với giá vào lệnh
stop_loss_pct = 0.05  # Cắt lỗ khi giá giảm 3% so với giá vào lệnh

# Khởi tạo biến cho giá Stop Loss và Take Profit
take_profit = pd.Series(index=data.index)
stop_loss = pd.Series(index=data.index)

# Tính Stop Loss và Take Profit
for i in range(1, len(data)):
    if entries.iloc[i]:  # Khi có tín hiệu mua
        take_profit.iloc[i] = data['close'].iloc[i] * (1 + take_profit_pct)
        stop_loss.iloc[i] = data['close'].iloc[i] * (1 - stop_loss_pct)
    elif exits.iloc[i]:  # Khi có tín hiệu bán
        take_profit.iloc[i] = None
        stop_loss.iloc[i] = None

# Tín hiệu mua/bán
entries = data['ema_short'] > data['ema_long']  # EMA9 > EMA21 -> Mua
exits = data['ema_short'] < data['ema_long']   # EMA9 < EMA21 -> Bán

# Thêm điều kiện Stop Loss và Take Profit
# Khi giá đóng cửa vượt qua Take Profit hoặc Stop Loss, thoát khỏi lệnh
stop_loss_trigger = data['close'] < stop_loss  # Cắt lỗ nếu giá thấp hơn Stop Loss
take_profit_trigger = data['close'] > take_profit  # Chốt lời nếu giá cao hơn Take Profit

# Xác định khi nào sẽ đóng lệnh
exit_signal = exits | stop_loss_trigger | take_profit_trigger

# Chạy backtest
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exit_signal,
    size=1,
    fees=0.001,
    freq="1D"
)

# Hiển thị kết quả
print(portfolio.stats())

# Vẽ biểu đồ backtest
portfolio.plot().show()

# Lưu kết quả vào file CSV
portfolio.stats().to_csv('ema_9_21_with_stop_loss_take_profit.csv')
print("Các kết quả đã được lưu vào các file CSV.")


In [ ]:
import vectorbt as vbt
import pandas as pd
import vnstock
import datetime as dt
from vnstock import Vnstock

# Lấy dữ liệu chứng khoán

# Tính Typical Price
typical_price = (data['high'] + data['low'] + data['close']) / 3

# Tính True Range (TR)
high_low = data['high'] - data['low']
high_close = (data['high'] - data['close'].shift(1)).abs()
low_close = (data['low'] - data['close'].shift(1)).abs()

true_range = pd.concat([high_low, high_close, low_close], axis=1)
true_range = true_range.max(axis=1)

# Tính ATR (Average True Range) với cửa sổ 14 ngày
atr = true_range.rolling(window=14).mean()

# Tính EMA của Typical Price
ema_typical_price = typical_price.ewm(span=20).mean()

# Tính Keltner Channel
keltner_upper = ema_typical_price + (atr * 1.5)  # Dải trên
keltner_lower = ema_typical_price - (atr * 1.5)  # Dải dưới

# Tín hiệu mua/bán
entries = data['close'] > (keltner_upper)  # Vượt dải trên -> Mua
exits = data['close'] < (keltner_lower)    # Xuống dải dưới -> Bán

# Thêm Take Profit và Stop Loss
take_profit_pct = 0.1  # Chốt lời khi giá tăng 5% so với giá vào lệnh
stop_loss_pct = 0.05   # Cắt lỗ khi giá giảm 3% so với giá vào lệnh

# Khởi tạo biến cho giá Stop Loss và Take Profit
take_profit = pd.Series(index=data.index)
stop_loss = pd.Series(index=data.index)

# Tính Stop Loss và Take Profit
for i in range(1, len(data)):
    if entries.iloc[i]:  # Khi có tín hiệu mua
        take_profit.iloc[i] = data['close'].iloc[i] * (1 + take_profit_pct)
        stop_loss.iloc[i] = data['close'].iloc[i] * (1 - stop_loss_pct)
    elif exits.iloc[i]:  # Khi có tín hiệu bán
        take_profit.iloc[i] = None
        stop_loss.iloc[i] = None

# Điều kiện Thoát Lệnh (Take Profit và Stop Loss)
stop_loss_trigger = data['close'] < stop_loss  # Cắt lỗ nếu giá thấp hơn Stop Loss
take_profit_trigger = data['close'] > take_profit  # Chốt lời nếu giá cao hơn Take Profit

# Xác định khi nào sẽ đóng lệnh
exit_signal = exits | stop_loss_trigger | take_profit_trigger

# Tạo portfolio với tín hiệu mua/bán
portfolio = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries,
    exits=exit_signal,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# In ra các thống kê
print(portfolio.stats())

# Vẽ biểu đồ
portfolio.plot().show()

# Lưu kết quả vào file CSV
portfolio.stats().to_csv('keltner_channel_with_tp_sl.csv')
print("Các kết quả đã được lưu vào các file CSV.")


In [ ]:
import pandas as pd
import glob

# Đường dẫn tới thư mục chứa các file CSV
path = "./"
csv_files = glob.glob(path + "*.csv")

strategies = {}
for file in csv_files:
    strategy_name = file.split('/')[-1].replace('.csv', '')
    df = pd.read_csv(file)
    
    # Kiểm tra nếu file có định dạng thống kê chiến lược
    if 'Unnamed: 0' in df.columns and len(df.columns) == 2:
        stats = df.set_index('Unnamed: 0')  # Đặt 'Unnamed: 0' làm chỉ số
        value_column = [col for col in stats.columns if col != 'Unnamed: 0'][0]  # Lấy cột giá trị ('0' hoặc 'close')
        strategies[strategy_name] = stats[value_column]
    else:
        print(f"Bỏ qua file {file} - không phải định dạng thống kê chiến lược.")

# Chuyển thành DataFrame
comparison_df = pd.DataFrame(strategies)
print(comparison_df)

In [ ]:
# Chọn các chỉ số quan trọng
key_metrics = [
    'Total Return [%]', 
    'Sharpe Ratio', 
    'Max Drawdown [%]', 
    'Win Rate [%]', 
    'Profit Factor',
    'Calmar Ratio',
    'Omega Ratio',
    'Sortino Ratio'
]

# Lọc DataFrame
filtered_comparison = comparison_df.loc[key_metrics]
print(filtered_comparison)

# Chuyển sang kiểu số (nếu cần)
filtered_comparison = filtered_comparison.astype(float)

# Lưu bảng so sánh
filtered_comparison.to_csv('strategy_comparison.csv')
print("Bảng so sánh đã được lưu vào 'strategy_comparison.csv'.")

# Trực quan hóa với matplotlib
import matplotlib.pyplot as plt

# Chọn dữ liệu và các chiến lược từ bảng filtered_comparison
strategies = filtered_comparison.columns
values = filtered_comparison.loc['Total Return [%]']

# Vẽ biểu đồ cột với width tùy chỉnh và khoảng cách rộng hơn
plt.bar(strategies, values, width=0.6)  # Điều chỉnh width để thay đổi khoảng cách giữa các cột

# Điều chỉnh nhãn của trục hoành
plt.ylabel('Total Return [%]')
plt.xticks(rotation=45)  # Xoay nhãn trục hoành 45 độ

# Thêm tiêu đề
plt.title('Total Return Comparison')

# Hiển thị đồ thị
plt.show()

In [ ]:
import pandas as pd

# Lọc các chiến lược có Total Return [%] > 1000
filtered_comparison_above_1000 = filtered_comparison.loc['Total Return [%]']
filtered_comparison_above_1000 = filtered_comparison_above_1000[filtered_comparison_above_1000 > 1000]

# Sắp xếp từ cao xuống thấp
filtered_comparison_above_1000_sorted = filtered_comparison_above_1000.sort_values(ascending=False)

# Lọc bảng so sánh theo các chiến lược có return > 1000
comparison_above_1000_sorted = comparison_df[filtered_comparison_above_1000_sorted.index]

# Hiển thị bảng trong Jupyter Notebook
import IPython.display as display
display.display(comparison_above_1000_sorted)

# Xuất bảng đã lọc ra file CSV
comparison_above_1000_sorted.to_csv('filtered_strategies_above_1000.csv')

print("Bảng đã được lưu vào 'filtered_strategies_above_1000.csv'.")


In [ ]:
import vectorbt as vbt
import pandas as pd

# Lấy dữ liệu cổ phiếu
vnstock_instance = Vnstock().stock(symbol='ACB', source='TCBS')

stock = "HPG"
end_date = dt.datetime.now().strftime('%Y-%m-%d')
start_date = (dt.datetime.now() - dt.timedelta(days=7000)).strftime('%Y-%m-%d')
data = vnstock_instance.quote.history(symbol=stock, start=start_date, end=end_date, interval='1D')

data.set_index("time", inplace=True)
data.index = pd.to_datetime(data.index)

# Giả sử data['close'] đã được định nghĩa (dữ liệu giá đóng cửa)
# Tính MA10, MA50, MA200
ma10 = vbt.MA.run(data['close'], window=10).ma
ma50 = vbt.MA.run(data['close'], window=50).ma
ma200 = vbt.MA.run(data['close'], window=200).ma

# Tín hiệu mua/bán (MA50 cắt lên MA200 -> Mua, MA50 cắt xuống MA200 -> Bán)
entries_ma = (ma50 > ma200) # Mua khi MA50 cắt lên MA200
exits_ma = (ma50 < ma200)   # Bán khi MA50 cắt xuống MA200

# Tạo portfolio với tín hiệu mua/bán
portfolio_ema = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries_ema,
    exits=exits_ema,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)


# Lấy giá nhập lệnh (entry price) từ portfolio
entry_price = pd.Series(index=data['close'].index, dtype=float)  # Tạo Series rỗng với cùng index
buy_orders = portfolio_ma.orders.records_readable[
    portfolio_ma.orders.records_readable['Side'] == 'Buy'
]
for idx, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()  # Điền giá nhập lệnh cho các thời điểm sau

# Tính toán Stop Loss và Take Profit dựa trên giá nhập lệnh
stop_loss_level = entry_price * (1 - stop_loss_pct)  # Mức giá Stop Loss
take_profit_level = entry_price * (1 + take_profit_pct)  # Mức giá Take Profit

# Tín hiệu Stop Loss và Take Profit
stop_loss = data['close'] < stop_loss_level  # Giá giảm xuống dưới mức SL
take_profit = data['close'] > take_profit_level  # Giá tăng vượt mức TP

# Kết hợp tín hiệu thoát lệnh: MA50 < MA200 hoặc SL hoặc TP
final_exits = exits_ma | stop_loss | take_profit

# Tạo portfolio cuối cùng với SL và TP
portfolio_ma_final = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries_ma,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True

)

# In ra các thống kê
print(portfolio_ma_final.stats())

# Lưu kết quả vào CSV và vẽ biểu đồ
portfolio_ma_final.stats().to_csv('ma200_ma50_with_stoploss_takeprofit.csv')
portfolio_ma_final.plot().show()
print("Chiến lược MA50 cắt lên MA200 với Stop Loss và Take Profit đã được lưu vào file CSV.")

In [ ]:
import vectorbt as vbt
import pandas as pd
import datetime as dt

# Lấy dữ liệu chứng khoán

# Tính EMA50 và EMA200 bằng pandas
ema200 = data['close'].ewm(span=200, adjust=False).mean()
ema50 = data['close'].ewm(span=50, adjust=False).mean()

# Tín hiệu mua/bán (EMA50 cắt lên EMA200 -> Mua, EMA50 cắt xuống EMA200 -> Bán)
entries_ema = ema50 > ema200
exits_ema = ema50 < ema200

# Tạo portfolio với tín hiệu mua/bán
portfolio_ema = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries_ema,
    exits=exits_ema,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# Lấy giá nhập lệnh
entry_price = pd.Series(index=data['close'].index, dtype=float)
buy_orders = portfolio_ema.orders.records_readable[
    portfolio_ema.orders.records_readable['Side'] == 'Buy'
]
for idx, row in buy_orders.iterrows():
    entry_price.loc[row['Timestamp']] = row['Price']
entry_price = entry_price.ffill()

# Tính toán Stop Loss và Take Profit
stop_loss_pct = 0.05  # 5%
take_profit_pct = 0.1 # 10%
stop_loss_level = entry_price * (1 - stop_loss_pct)
take_profit_level = entry_price * (1 + take_profit_pct)

# Tín hiệu Stop Loss và Take Profit
stop_loss = data['close'] < stop_loss_level
take_profit = data['close'] > take_profit_level

# Kết hợp tín hiệu thoát lệnh
final_exits = exits_ema | stop_loss | take_profit

# Tạo portfolio cuối cùng
portfolio_final = vbt.Portfolio.from_signals(
    close=data['close'],
    entries=entries_ema,
    exits=final_exits,
    size=1,
    fees=0.001,
    freq="1D",
    accumulate=True
)

# In kết quả
print(portfolio_final.stats())
portfolio_final.stats().to_csv('ema50_ema200_sl_tp.csv')
portfolio_final.plot().show()
print("Chiến lược EMA50 cắt EMA200 với SL/TP đã được lưu vào file CSV.")

In [ ]:
jupyter nbconvert --ClearOutputPreprocessor.enabled=True --inplace your_notebook.ipynb
